In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set_theme()

In [ ]:
def plotDataStackBar(file, nIter, title, colsToUse):
    fig, (ax1) = plt.subplots(1,1)
    dataMaxMeans = buildData(file, nIter, colsToUse, True)
    #dataMeanMeans = buildData(file, nIter, colsToUse, False)
    buildAx(title + " - Max time", ax1, dataMaxMeans)
    #buildAx(title + " - Mean time", ax2, dataMeanMeans)
   
def buildData(file, nIter, colsToUse, isMax):
    data = pd.read_csv(file,header=0,sep=";")
    data = data[::2] if isMax else data[1::2]
    dataMeans = pd.DataFrame([data[i:i+nIter].mean() for i in range(0, len(data), nIter)])
    dataMeans = dataMeans[colsToUse]
    dataMeans = dataMeans.reindex(dataMeans.mean().sort_values(ascending=False).index, axis=1)
    return dataMeans
        
def buildAx(title, ax, data):
    xTicks = [2**i for i in range(len(data))] 
    ax.set_xscale("log", base=2)
    bar_data = np.array([data[col] for col in data.columns])
    widths = np.diff(xTicks + [xTicks[-1] * 2])*0.4
    ax.bar(xTicks, bar_data[0], widths, label=data.columns[0])
    for i in range(1, bar_data.shape[0]):
        ax.bar(xTicks, bar_data[i], widths, bottom=np.sum(bar_data[:i], axis=0), label=data.columns[i])
    xLabels = [str(i) for i in xTicks]
    ax.set_xticks(xTicks, xLabels)
    y_ticks = ax.get_yticks()
    ax.set_ylim(0, max(y_ticks))
    ax.set_xlabel("Number of MPI tasks")
    ax.set_ylabel("Time (s)")
    ax.set_title(title)
    ax.legend(loc="upper right")
    # ax_twin = ax.twinx()
    # scalability = [[data[col].iloc[0] / data[col].iloc[j] for j in range(len(xTicks))] for col in data.columns]
    # for i in range(len(scalability)):
    #     ax_twin.plot(xTicks, scalability[i])
    # ax_twin.set_yticks(y_ticks)
    # maxScal = round(np.max(scalability)+0.01, 2)
    # y_ticks_scal = [maxScal/(len(y_ticks)-1)*i for i in range(len(y_ticks))]
    # ax_twin.set_ylim(0, maxScal)
    # ax_twin.set_yticks(y_ticks_scal)
    # ax_twin.grid(False)
    # ax_twin.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    # ax_twin.set_ylabel('Scalability')
    ax.plot()


In [ ]:
# "initacc", "copyin", "init", "update", "sendrecv", "evolve", "save", "copyout", "total"
colsToUse = [ "init", "update", "sendrecv"]
plotDataStackBar("cpu12000.csv", 1, "CPU, size 12000, 10 iter", colsToUse)

In [ ]:
def read_data(files, cols_to_use):
    data_frames = []
    for file in files:
        df = pd.read_csv(file, usecols=cols_to_use, header=0, sep=";")
        data_frames.append(df)
    return data_frames

def plot_stack_bar(data_frames, titles):
    fig, ax = plt.subplots(1, 1)
    fig.set_figheight(5)
    fig.set_figwidth(10)    
    datacpu12001 = data_frames[0].loc[0].to_frame().T
    datagpu12001 = data_frames[1].loc[0].to_frame().T
    datacpu120001 = data_frames[2].loc[0].to_frame().T
    datagpu120001 = data_frames[3].loc[0].to_frame().T
    datagpu400008 = data_frames[4].loc[10].to_frame().T
    dfs = [datacpu12001, datagpu12001, datacpu120001, datagpu120001, datagpu400008]
    combined_df = pd.concat(dfs, ignore_index=True)
    x_ticks = np.arange(len(dfs))
    bottom = np.zeros(combined_df.shape[0])
    for col in combined_df.columns:
        ax.bar(x_ticks, combined_df[col], bottom=bottom, width=0.8, label=col)
        bottom += combined_df[col]
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(titles)
    ax.set_ylabel("Time")
    ax.set_title("Comparison between CPU and GPU for different sizes and 1 node")
    ax.legend(loc='upper left')
    plt.show()

files = ['cpu1200.csv','gpu1200.csv', 'cpu12000.csv', 'gpu12000.csv', 'gpu40000.csv'] 
cols_to_use = ["initacc","init", "update", "sendrecv", "copyout"] 
titles = ['CPU, 1200-10', 'GPU, 1200-10', 'CPU, 12000-10', 'GPU, 12000-10', 'GPU, 40000-1000\n(8 nodes)']

plot_stack_bar(read_data(files, cols_to_use), titles)